In [ ]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [ ]:
# Clear any logs from previous runs
!rm -rf /app/logs/

In [1]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from src.model import Sampling
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

2024-07-22 07:41:45.381299: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 07:41:45.384851: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 07:41:45.396130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 07:41:45.413220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 07:41:45.418005: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 07:41:45.432292: I tensorflow/core/platform/cpu_feature_gu

In [2]:
#================== initialization ==================
currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "testVAEModel"
LATENT_DIM = 16
VAE_LR = 0.00001
EPOCHS = 1
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
DISPLAY = False
WANDB_LOG = False
RESUME = False
log_dir = f"/app/logs/{PROJECT}/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
Scaler = joblib.load(SCALER_PATH )

In [94]:
notes = f"""
test Run use_bias set false , no tranfrom
"""
import wandb
from wandb.integration.keras import WandbCallback
CONFIG = {"latent_dim":LATENT_DIM,
          "learning_rate": VAE_LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "VAE",
          "dataset": "OptionsChaine",
          "encoder_dense_units":[64,32],
          "encoder_dropout_rate":0.2,
          "decoder_dense_units":[32, 64],
          "decoder_dropout_rate":0.2,
          "use_bias":False,
          "transform":True
           }

if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

In [4]:
#Example

# from IPython.display import clear_output,display, HTML
# import numpy as np
# #load scaler
# scaler = MinMaxScaler()
# PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
# random.shuffle(PartitionDate)
# scaler = joblib.load(SCALER_PATH)


# for i,partdate in enumerate(PartitionDate) :
#     df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
#                                  , filters=[('PartitionDate', '=', partdate)]
#                                 )
#     df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
#     df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
#     DATA  = np.empty((0,) + (20,9) ) 
#     for opt_id in np.unique( df[["OPTIONS_ID"]].values):
#         df_filter  = df[df["OPTIONS_ID"]==opt_id]
#         if len(df_filter) == 20:
#             DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
#         else:
#             #print( len(df_filter) )
#             #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
#             pass
            
#     ## Save the NumPy array to an HDF5 file
#     # with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
#     #     dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

#     print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [5]:
# # Save the NumPy array to an HDF5 file
# with h5py.File(H5_PATH, 'w') as f:
#     #dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')
#     #test
#     dset = f.create_dataset('dataset', data=DATA, compression='gzip')

In [6]:
#=====================================================================

In [7]:
#SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
select_x = [i for i,c in  enumerate(SCALER_COL) if c in ['DTE','INTRINSIC_VALUE'] ]
select_y = [i for i,c in enumerate(SCALER_COL) if c in ['C_BID',	'C_ASK',  'P_BID',	'P_ASK'] ]

In [8]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (16,len(select_x) ), 
            dense_units = CONFIG["encoder_dense_units"], 
            dropout_rate= CONFIG["encoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= (16,len(select_y) ),
            dense_units = CONFIG["decoder_dense_units"],
            dropout_rate= CONFIG["decoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           )
)



def dummy_loss(y_true, y_pred):
    return 0.0
    
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer )#, loss=dummy_loss)

In [9]:
model.encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (32, 16, 2)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (32, 2)           │          0 │ features[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (32, 64)          │        128 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (32, 64)          │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (32, 32)          │      2,048 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (32, 32)          │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (32, 16)          │        512 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_var (Dense)     │ (32, 16)          │        512 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,200 (12.50 KB)

 Trainable params: 3,200 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 4)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,656 (26.00 KB)

 Trainable params: 6,656 (26.00 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
################## show model ######################
if DISPLAY :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [12]:
## Example usage:
## Display the encoder model with reduced size
if DISPLAY :
    display_model(model.encoder, width=1024, height=512)

In [13]:
if DISPLAY :
    display_model(model.decoder, width=2500, height=512)

In [14]:
#================== loadmodel ====================

In [15]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
else:
    model.encoder = load_model(model_path+'/'+f'encoder.keras') 
    model.decoder = load_model(model_path+'/'+f'decoder.keras') 

In [ ]:
# ================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)

STOP_MODEL = False
STACK_DATA = np.empty((0,) + (16,9) ) #init STACK_DATA

for partdate in PartitionDate[:5] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)
    DATA = np.vstack((DATA ,STACK_DATA))
    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        # if DATA.isna().sum().sum() > 0:
        #     print("Data contains NaNs. Please handle them before scaling.")
        STACK_DATA = np.empty((0,) + (16,9) )
        X = DATA[:, :, select_x]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
        Y = DATA[:, :, select_y]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) ,callbacks=[tensorboard_callback])
        #history = model.fit(x_train , y_train, epochs=5, batch_size=BATCH_SIZE )
        if  np.isnan(  np.average( history.history['kl_loss'] )  ) or np.isnan(  np.average( history.history['val_kl_loss'] )  ):
            STOP_MODEL = True 
    
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
    if STOP_MODEL :
        break
    
            
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
if WANDB_LOG : wandb.finish()

In [ ]:
`====================================================

In [ ]:
======================== predict =========================

In [129]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")
    if CONFIG['transform'] or True :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)

CONFIG['transform'] : True


In [130]:
X = DATA[:, :, select_x][:]
Y_real = DATA[:, :, select_y][:]
#===========
# X = x_train
# Y_real = y_train
# #===========
df = pd.DataFrame(
    DATA[:, :, :3][:1].reshape(16, 3), 
    columns=SCALER_COL[:3])
#print(df)

In [131]:
z_mean, log_var = model.encoder(X) 
z = Sampling()([z_mean, log_var])
decode_data = model.decoder(z)

In [132]:
col = [SCALER_COL[i] for i in select_x]
col += [SCALER_COL[i] for i in select_y]

In [140]:
#======= decode_data ==========
if CONFIG['transform'] :
    decode_transformed=Scaler.inverse_transform(
        tf.concat([X[0],np.full((16, 1), 1e-8), 
           decode_data[0][:,:2],np.full((16, 1), 1e-8),
           decode_data[0][:,2:],np.full((16, 1), 1e-8) ], axis=1)
    )
    deDF=pd.DataFrame(  
    decode_transformed
    ,columns=SCALER_COL)
else:
    deDF=pd.DataFrame(  
    tf.concat([X[0], decode_data.numpy()[0]], axis=-1).numpy() 
    ,columns=col)
#deDF[deDF <= 1e-8] = 0
deDF

,DTE,INTRINSIC_VALUE,TOTAL_VOLUME,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,-0.000002,-6.442553e-07,583.198416,106.475821,-45.744477,38.396628,43.013970,10.252804,59.252667
1,-0.000002,-6.442553e-07,583.198416,55.226707,-15.288197,38.396628,37.302756,14.352741,59.252667
2,-0.000002,-6.442553e-07,583.198416,-15.552384,20.106584,38.396628,8.914482,-8.963986,59.252667
3,-0.000002,-6.442553e-07,583.198416,81.284210,-16.862326,38.396628,20.460961,8.482422,59.252667
4,-0.000002,-6.442553e-07,583.198416,-34.783771,47.041647,38.396628,20.862258,40.457631,59.252667
5,7.039999,1.087000e+01,583.198416,4.297903,36.849526,38.396628,8.855051,70.928443,59.252667
6,7.039999,1.037000e+01,583.198416,-15.550674,64.189343,38.396628,20.494004,27.164844,59.252667
7,7.039999,9.870000e+00,583.198416,131.495488,18.884019,38.396628,43.492350,10.811196,59.252667
8,7.039999,9.370000e+00,583.198416,61.999548,77.893412,38.396628,16.802733,47.689937,59.252667
9,7.039999,8.870000e+00,583.198416,20.595835,59.518982,38.396628,-8.228351,69.388276,59.252667


In [141]:
#======= Y_real ==========
if CONFIG['transform'] :
    decode_transformed=Scaler.inverse_transform(
        tf.concat([X[0],np.zeros((16, 1)), 
           Y_real[0][:,:2],np.zeros((16, 1)),
           Y_real[0][:,2:],np.zeros((16, 1)) ], axis=1)
    )
    deDF=pd.DataFrame(  
    decode_transformed
    ,columns=SCALER_COL)
else:
    YDF=pd.DataFrame(tf.concat([X[0], Y_real[0]] , axis=-1),columns=col)
YDF

,DTE,INTRINSIC_VALUE,C_BID,C_ASK,P_BID,P_ASK
0,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
1,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
2,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
3,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
4,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
5,2.000000e+00,-4.153000e+01,1.000000e-02,2.000000e-02,4.120000e+01,4.155000e+01
6,2.000000e+00,-4.653000e+01,1.000000e-08,2.000000e-02,4.620000e+01,4.654000e+01
7,2.000000e+00,-5.153000e+01,1.000000e-08,2.000000e-02,5.120000e+01,5.155000e+01
8,2.000000e+00,-5.653000e+01,1.000000e-08,1.000000e-02,5.620000e+01,5.655000e+01
9,2.000000e+00,-6.153000e+01,1.000000e-08,1.000000e-02,6.119000e+01,6.154000e+01


In [ ]:
z_mean

In [ ]:
log_var

In [ ]:
encodeDF=pd.DataFrame(  
    tf.concat([z_mean,log_var,z], axis=-1).numpy().transpose(),
    columns=['z_mean','log_var','z'] )
encodeDF

In [ ]:
np.isnan(Y_real).sum()

In [ ]:
np.isnan(X).sum()

In [ ]:
np.isinf(Y_real).sum()

In [ ]:
np.isinf(X).sum()

In [ ]:
======================= _compute_loss ==============================

In [ ]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [ ]:
z_mean

In [ ]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

In [ ]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [ ]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [ ]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [ ]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

In [ ]:
========== kiras vae origi

In [ ]:
Y_real[0][0]

In [ ]:
 tf.concat(decode_data, axis=-1).numpy()[0][0]

In [ ]:
Y_real[0] -  tf.concat(decode_data, axis=-1).numpy()

In [ ]:
features_loss = tf.reduce_mean(
    tf.reduce_sum(
        tf.keras.losses.mean_squared_error(Y_real, tf.concat(decode_data, axis=-1)),
        axis=(1),
    )
)

In [ ]:
features_loss

In [ ]:
# features_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         keras.losses.categorical_crossentropy(features_real, features_gen),
#         axis=(1),
#     )
# )
# kl_loss = -0.5 * tf.reduce_sum(
#     1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1
# )
# kl_loss = tf.reduce_mean(kl_loss)

# property_loss = tf.reduce_mean(
#     keras.losses.binary_crossentropy(qed_true, qed_pred)
# )

# graph_loss = self._gradient_penalty(graph_real, graph_generated)

# return kl_loss + property_loss + graph_loss + adjacency_loss + features_loss

In [ ]:
======================= inverse_transform ========================

In [ ]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [ ]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [ ]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

In [ ]:
=====================================================================